# 使用 PAI Python SDK 训练和部署PyTorch模型

[PyTorch](https://pytorch.org/) 是当前主流的深度学习框架, 它提供了高度的灵活性和速度，使得开发者能够快速的构建和训练深度学习模型。PyTorch使用动态图技术，可以更好的实现复杂的计算流程，支持GPU加速，也可以方便的与其他Python库进行集成。当前示例中，我们将使用 PAI Python SDK，在 PAI 完成一个 PyTorch 模型的训练，然后使用训练获得的模型部署推理服务。主要流程包括:


1. 准备工作：

安装 PAI Python SDK，并完成 SDK 配置.

2. 准备训练脚本

我们基于 PyTorch 示例仓库中的 MNIST 训练脚本作为模版，演示如何编写一个符合要求的训练脚本。


3. 提交训练作业

使用 PAI Python SDK提供的 Estimator API，创建一个训练作业，提交到云上执行。

4. 部署模型

将以上训练作业输出的模型，分别使用 Processor 模式和镜像部署模式，部署到 PAI-EAS，创建在线推理服务。



## Step1: 准备工作

我们需要首先安装 PAI Python SDK 以运行本示例。

In [ ]:
!python -m pip install alipai -f https://pai-sdk.oss-cn-shanghai.aliyuncs.com/repo/index.html



SDK 需要配置访问阿里云服务需要的 AccessKey，以及当前使用的工作空间和OSS Bucket。在 PAI SDK 安装之后，通过在 **命令行终端** 中执行以下命令，按照引导配置密钥，工作空间等信息。


```shell

# 以下命令，请在 命令行终端 中执行.

python -m pai.toolkit.config

```

我们可以通过以下代码验证当前的配置。

In [ ]:
import pai
from pai.session import get_default_session

print(pai.__version__)

sess = get_default_session()

assert sess.workspace_name is not None
print(sess.workspace_name)

## Step2: 准备训练脚本

当前示例使用 MNIST 数据集训练一个图片分类模型，这里我们以 PyTorch 官方提供的 [MNIST 示例](https://github.com/pytorch/examples/blob/main/mnist/main.py) 为基础，修改了模型保存部分的逻辑，作为训练脚本。

修改代码如下，训练脚本需要将输出模型保存到 `/ml/output/model/` 路径下。

```diff

-     if args.save_model:
-         torch.save(model.state_dict(), "mnist_cnn.pt")

+     # 保存模型
+     save_model(model)
+
+
+ def save_model(model):
+     """将模型转为TorchScript，保存到指定路径."""
+
+     output_model_path = "/ml/output/model/"
+     os.makedirs(output_model_path, exist_ok=True)
+
+     m = torch.jit.script(model)
+     m.save(os.path.join(output_model_path, "mnist_cnn.pt"))

```

PAI 提供的预置 [PyTorch Processor](https://help.aliyun.com/document_detail/470458.html) 在创建服务时，要求输入的模型是 [TorchScript 格式](https://pytorch.org/docs/stable/jit.html) 。在当前示例中，我们将模型导出为 `TorchScript格式` ，然后使用 `PyTorch Processor` 创建推理服务。

用户可以根据需求使用 [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html) 直接保存模型，导出为 [ONNX格式](https://pytorch.org/docs/stable/onnx.html)等方式保存模型，脚本需要将模型写到 `/ml/output/model/` 目录下，PAI才能够将用户的模型持久化保存到 OSS。



完整的作业脚本如下。在 Notebook 中执行时，`%%writefile` 指令会将单元格内的代码保存到本地。

In [ ]:
!mkdir -p train_src

In [ ]:
%%writefile train_src/train.py

# source: https://github.com/pytorch/examples/blob/main/mnist/main.py
from __future__ import print_function

import argparse
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )
            if args.dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(
                output, target, reduction="sum"
            ).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )


def main():
    # Training settings
    parser = argparse.ArgumentParser(description="PyTorch MNIST Example")
    parser.add_argument(
        "--batch-size",
        type=int,
        default=64,
        metavar="N",
        help="input batch size for training (default: 64)",
    )
    parser.add_argument(
        "--test-batch-size",
        type=int,
        default=1000,
        metavar="N",
        help="input batch size for testing (default: 1000)",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=14,
        metavar="N",
        help="number of epochs to train (default: 14)",
    )
    parser.add_argument(
        "--lr",
        type=float,
        default=1.0,
        metavar="LR",
        help="learning rate (default: 1.0)",
    )
    parser.add_argument(
        "--gamma",
        type=float,
        default=0.7,
        metavar="M",
        help="Learning rate step gamma (default: 0.7)",
    )
    parser.add_argument(
        "--no-cuda", action="store_true", default=False, help="disables CUDA training"
    )
    parser.add_argument(
        "--dry-run",
        action="store_true",
        default=False,
        help="quickly check a single pass",
    )
    parser.add_argument(
        "--seed", type=int, default=1, metavar="S", help="random seed (default: 1)"
    )
    parser.add_argument(
        "--log-interval",
        type=int,
        default=10,
        metavar="N",
        help="how many batches to wait before logging training status",
    )
    parser.add_argument(
        "--save-model",
        action="store_true",
        default=False,
        help="For Saving the current Model",
    )
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    train_kwargs = {"batch_size": args.batch_size}
    test_kwargs = {"batch_size": args.test_batch_size}
    if use_cuda:
        cuda_kwargs = {"num_workers": 1, "pin_memory": True, "shuffle": True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
    dataset1 = datasets.MNIST("../data", train=True, download=True, transform=transform)
    dataset2 = datasets.MNIST("../data", train=False, transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    # 保存模型
    save_model(model)


def save_model(model):
    """将模型转为TorchScript，保存到指定路径."""

    output_model_path = "/ml/output/model/"
    os.makedirs(output_model_path, exist_ok=True)

    m = torch.jit.script(model)
    m.save(os.path.join(output_model_path, "mnist_cnn.pt"))


if __name__ == "__main__":
    main()

PAI 提供了常见的机器学习框架的基础镜像，支持用户提交训练作业。通过 `pai.image.list_images` 可以列出PAI提供的公共镜像


In [ ]:
from pai.image import list_images, ImageScope, retrieve

print("FrameworkVersion\tAcceleratorType\tImageURI")
for image in list_images(framework_name="PyTorch", image_scope=ImageScope.TRAINING):
    print(
        "{}\t{}\t{}".format(
            image.framework_version, image.accelerator_type, image.image_uri
        )
    )

FrameworkVersion	AcceleratorType	ImageURI
1.12PAI	gpu	registry.cn-hangzhou.aliyuncs.com/pai-dlc/pytorch-training:1.12PAI-gpu-py38-cu113-ubuntu20.04
1.8PAI	gpu	registry.cn-hangzhou.aliyuncs.com/pai-dlc/pytorch-training:1.8PAI-gpu-py36-cu101-ubuntu18.04
1.7PAI	gpu	registry.cn-hangzhou.aliyuncs.com/pai-dlc/pytorch-training:1.7PAI-gpu-py36-cu101-ubuntu18.04


在当前示例中，我们使用PAI提供的`1.8PAI`版本的GPU镜像提交训练作业。

In [ ]:
image_uri = retrieve(
    "PyTorch", framework_version="1.8PAI", accelerator_type="GPU"
).image_uri
print(image_uri)

registry.cn-hangzhou.aliyuncs.com/pai-dlc/pytorch-training:1.8PAI-gpu-py36-cu101-ubuntu18.04


## Step3: 提交训练作业

`Estimator` 支持用户将本地的训练脚本，提交到 PAI，使用指定的作业镜像执行，使用云上的资源执行训练作业。

- 用户通过 `entry_point` 和  `source_dir` 指定训练脚本:

`source_dir` 目录是本地执行脚本所在的目录，对应的目录会被打包上传到用户的OSS Bucket，在训练作业执行前，加载到作业的运行环境中。 `entry_point` 是训练作业的启动脚本，支持使用 Python 或是 Shell 文件。

对于 Python 脚本，训练作业会以通过以下默认启动:

```shell

python {entry_point} --{hyperparameters_in_arguments}

```

对于 Shell 脚本，则会使用以下方式运行对应的脚本

```shell

sh -s {entry_point} --{hyperparameters_in_arguments}

```


- 通过 `image_uri` 指定作业的训练镜像:

- 通过 `hyperparameters` 传递的作业使用的超参:

超参会通过命令行 arguments 的方式传递给到训练脚本。 例如以下示例中，对应的训练脚本的启动命令为:

```shell

python train.py --epochs 5 --batch-size 256

```

- 使用 `metric_definitions` 指定需要采集的Metric:

PAI 的训练服务从训练作业输出日志中，以正则的方式捕获用户指定的Metrics信息。用户可以通过作业详情页查看输出日志。

- 使用 `instance_type` 指定作业使用的机器实例类型:


对于提交训练作业的更加详细的介绍，请查看 [文档:提交训练作业](https://pai-sdk.oss-cn-shanghai.aliyuncs.com/pai/doc/latest/user-guide/estimator.html)

In [ ]:
from pai.estimator import Estimator
from pai.image import retrieve


est = Estimator(
    entry_point="train.py",
    source_dir="./train_src/",
    image_uri=image_uri,
    instance_type="ecs.gn6i-c4g1.xlarge",  # 4vCPU 15GB 1*NVIDIA T4
    hyperparameters={
        "epochs": 5,
        "batch-size": 64 * 4,
    },
    metric_definitions=[
        {
            "Name": "loss",
            "Regex": r".*loss=" r"([-+]?[0-9]*.?[0-9]+(?:[eE][-+]?[0-9]+)?).*",
        },
    ],
    base_job_name="pytorch_mnist",
)
est.fit()

# 训练作业产出的模型路径
print("TrainingJob output model data:")
print(est.model_data())


`est.fit` 方法将用户的训练作业提交到 PAI 上执行。任务提交之后，`.fit` 方法会打印作业的URL，以及作业的执行日志，默认等待到作业执行结束才会退出。

当用户需要直接使用 OSS 上的数据，或是 NAS 盘上的数据，可以通过 `.fits` 方法的 `inputs` 参数传递。

```python

est.fit(
	inputs={
		# 数据会被挂载到 /ml/input/data/train/ 路径下
		"train": "oss://{YourOssBucket}/path/to/train/data",	
		# 数据会被挂载到 /ml/input/data/test/ 路径下
		"test": "nas://{YourOssBucket}/path/to/test/data",	
	}
)

```

通过 `inputs` 传递数据存储路径会被挂载到 `/ml/input/data/{Name}` 目录下，用户的训练脚本可以通过读取对应的目录获得输入的数据。

## Step4: 部署推理服务

通过 `Estimator.model_data()`，我们可以拿到训练作业产出模型的OSS路径。下面的流程中我们将训练产出的模型部署到 PAI 创建在线推理服务。

部署推理服务的主要流程包括：

- 通过 `InferenceSpec` 描述如何使用模型构建推理服务。

用户可以选择使用 Processor 模式，或是自定义镜像的模式进行模型部署。以下示例中将使用两种方式分别部署获得的 PyTorch 模型。

- 通过 `Model.deploy` 方法，配置服务的使用资源，服务名称，等信息，创建推理服务。

对于部署推理服务的详细介绍，可以见: [文档:部署推理服务](https://pai-sdk.oss-cn-shanghai.aliyuncs.com/pai/doc/latest/user-guide/model.html)

### Processor 模式部署

[Processor](https://help.aliyun.com/document_detail/111029.html) 是 PAI 对于推理服务程序包的抽象描述，他负责加载模型，启动模型推理服务。模型推理服务会暴露 API 支持用户进行调用。

对于 PyTorch ，PAI提供了预置的 [PyTorch Processor](https://help.aliyun.com/document_detail/470458.html) ，用户可以方便得将获得的 `TorchScript` 格式的模型部署到 PAI，创建推理服务。


In [ ]:
from pai.model import Model, InferenceSpec
from pai.predictor import Predictor
from pai.common.utils import random_str


m = Model(
    model_data=est.model_data(),
    # 使用PAI提供的PyTorch Processor
    inference_spec=InferenceSpec(processor="pytorch_cpu_1.10"),
)

p: Predictor = m.deploy(
    service_name="tutorial_pt_mnist_proc_{}".format(random_str(6)),
    instance_type="ecs.c6.xlarge",
)

print(p.service_name)
print(p.service_status)

`Model.deploy` 返回的 `Predictor` 对象指向新创建的推理服务，可以通过 `Predictor.predict` 方法发送预测请求给到服务，拿到预测结果。


导出的TorchScritp模型要求输入为 Float32，数据格式格式的形状为 (BatchSize, Channel, Weight, Height)。我们使用 numpy 构建了一个测试样本数据，发送给到推理服务。

In [ ]:
import numpy as np

dummy_input = np.random.rand(2, 1, 28, 28).astype(np.float32)

# np.random.rand(1, 1, 28, 28).dtype
res = p.predict(dummy_input)
print(res)

print(np.argmax(res, 1))

在测试完成之后，删除相关的服务，释放资源。

In [ ]:
p.delete_service()

### 镜像部署

Processor 模式拉起的推理服务性能较高，适合于对于性能较为敏感的场景。对于一些需要灵活自定义的场景，例如用户模型使用了一些第三方的依赖，或是推理服务需要有前处理和后处理，可以通过镜像部署的方式的方式实现。 SDK 提供了便利方法 `InferenceSpec.from_serving_script` ，支持用户使用本地脚本配合 PAI 提供的基础镜像的方式创建推理服务。


以下是使用 Flask 编写的模型服务：在接受用户的HTTP请求，将数据图片进行预处理，然后使用模型进行推理，然后返回预测结果。


In [ ]:
!mkdir -p infer_src

In [ ]:
%%writefile infer_src/run.py


import json
from flask import Flask, request
from PIL import Image
import os
import torch
import torchvision.transforms as transforms
import numpy as np
import io

app = Flask(__name__)
# 用户指定模型，默认会被加载到当前路径下。 
MODEL_PATH = "/eas/workspace/model/"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.jit.load(os.path.join(MODEL_PATH, "mnist_cnn.pt")).to(device)

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)


@app.route("/", methods=["POST"])
def predict():
    # 预处理图片数据
    im = Image.open(io.BytesIO(request.data))
    input_tensor = transform(im).to(device)

    input_tensor.unsqueeze_(0)

    # 使用模型进行推理
    output_tensor = model(input_tensor)
    pred_res =output_tensor.detach().cpu().numpy()[0] 

    return json.dumps(pred_res.tolist())


if __name__ == '__main__':
    app.run(host="0.0.0.0", port=int(os.environ.get("LISTENING_PORT", 8000)))

# 使用测试数据集发送



通过 `InferenceSpec.from_serving_script`，我们基于本地脚本和 PAI 提供的 PyTorch 镜像创建了一个InferenceSpec对象。

- 用户指定的本地脚本目录 `source_dir` 的会被上传到OSS，然后挂载到容器作业中供用户推理程序使用。
  
- 推理服务的容器内会通过 ``python {{entry_point}}`` 的方式启动用户的推理程序。当前示例下，也就是以上的Flask 程序。

- PAI 提供了基础的推理镜像支持用户使用，用户可以通过 `pai.image.retrieve` 方法，指定参数 `image_scope="inference"` 获取指定框架在推理场景使用的镜像。

- 用户的代码或是模型的依赖，可以通过 `requirements` 参数指定，相应的依赖会在服务程序启动前被安装到环境中。



In [ ]:
from pai.model import InferenceSpec
from pai.image import retrieve, ImageScope

torch_image_uri = retrieve(
    framework_name="pytorch", framework_version="1.8PAI", accelerator_type="GPU"
).image_uri

inf_spec = InferenceSpec.from_serving_script(
    entry_point="run.py",
    source_dir="./infer_src/",
    image_uri=torch_image_uri,
    requirements=["flask==2.0.0"],
)
print(inf_spec.to_dict())

使用训练作业输出的模型，以及以上的 InferenceSpec，我们将通过 Model.deploy API部署一个在线推理服务。

In [ ]:
from pai.model import Model
from pai.common.utils import random_str
import numpy as np


m = Model(
    model_data=est.model_data(),
    inference_spec=inf_spec,
)

container_serving_predictor = m.deploy(
    service_name="torch_mnist_script_container_{}".format(random_str(6)),
    instance_type="ecs.gn6i-c4g1.xlarge",  # 4vCPU 15GB 1*NVIDIA T4
)


我们准备一张 MNIST 测试图片，用于发送给到推理服务。


In [ ]:

!pip install -q pillow


import base64
from PIL import Image
from IPython import display
import io


# raw_data是一张MNIST图片
raw_data = base64.b64decode(b"/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+rVhpmoarP5GnWNzeTYz5dvE0jfkoJovNMv8ATmK3tjc2zByhE8TIQw6jkdR6VVq9oumPrWuWGlxyLG95cRwK7dFLMFyfzr3aXwp4ltAfB3gWwudI01JNuoa7eZhku5AMHafvFOw2Dn6ZJ4z4yeLk1HUbXwrZSSy2Oh5heeaQu88wG1mLHk4wR9c+1eXUqsVYMpIIOQR2r1D4QazqOs/FnSG1fVLi9ZI5vL+2TNKc+U2ApYnB7/hXml5LLNfXEsxLSvIzOSMEsTk1DRVnT7+60vULe/spmhureQSRSL1Vh0NWNd1mXX9ZuNUuLe2gmuCGkS2QohbABbBJwTjJ9yelZ1f/2Q==")

im = Image.open(io.BytesIO(raw_data))

display.display(im)


推理服务使用 HTTP 请求体内的数据作为输入的图片，SDK 的 `raw_predict` 方法接受 bytes 数据类型的请求，通过 POST 方法，在请求内带上用户推理数据，发送给到推理服务。

In [ ]:
import json

resp = container_serving_predictor.raw_predict(data=raw_data)

print(json.loads(resp))

print(np.argmax(json.loads(resp)))

测试完成之后删除服务。

In [ ]:
container_serving_predictor.delete_service()